In [1]:
import os

if not os.path.isdir("/content/Separate Corpuses/FullCorpus/") or not os.path.isdir("/joey_experiments/"):
    !gsutil -m cp -r "gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/" .
    !gsutil -m cp -r "gs://mytranslator-298419-vcm/joey_experiments/" .

Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tc.model...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tok.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tok.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tc.model...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tok.txt_true.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tok.txt_true.txt...
- [8/8 files][651.5 MiB/651.5 MiB] 100% Done                                

In [2]:
import os
from os import path
source_language = "ru"
target_language = "be"
lang_pair = source_language+target_language
tag = "new_full"
model_type = "bpe"

use_cuda = False

if model_type == "bpe":
    # Learn BPEs on the training data.
    number_of_splits = 5000
    os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language)
    bpe_codes_file = "bpe.codes."+str(number_of_splits)

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["lang_pair"] = lang_pair
os.environ["tag"] = tag
os.environ["model_type"] = model_type

gdrive_path = "/content/joey_experiments/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = gdrive_path
!echo $gdrive_path

/content/joey_experiments/ru-be-new_full


In [3]:
raw_source_file = r"/content/FullCorpus/Corpus_Rus.txt"
raw_target_file = r"/content/FullCorpus/Corpus_Bel.txt"

# They should both have the same length.
! wc -l "$raw_source_file"
! wc -l "$raw_target_file"

429479 /content/FullCorpus/Corpus_Rus.txt
429479 /content/FullCorpus/Corpus_Bel.txt


In [4]:
%%capture
! pip install sacremoses

In [5]:
def preprocess(filepath, language, force_preprocess=False, pretrained_tc_model=None):
  tok_file = filepath+"_tok.txt"
  tc_model = pretrained_tc_model if pretrained_tc_model else filepath+"_tc.model"
  truecased_file = tok_file+"_true.txt"

  if force_preprocess:
    ! sed -i 's/"//g' "$filepath"
    ! sacremoses tokenize -l language -e 'utf-8' < "$filepath" > "$tok_file"
    if not pretrained_tc_model:
      ! sacremoses train-truecase -m "$tc_model" -j 4 < "$tok_file"
    ! sacremoses truecase -m "$tc_model" -j 4 < "$tok_file" > "$truecased_file"

  return truecased_file, tc_model

In [6]:
# Change the pointers to our files such that we continue to work with the tokenized and truecased data.
source_file, source_tc_model = preprocess(raw_source_file, source_language)
target_file, target_tc_model = preprocess(raw_target_file, target_language)

In [7]:
%%capture
# installing cuda
if use_cuda:
    !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
    !sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
    !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
    !sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
    !sudo apt-get update
    !sudo apt-get -y install cuda

In [8]:
# #Let's take a look what preprocessing did to the text.
# ! head "$raw_source_file"*
# ! head "$raw_target_file"*

In [9]:
# # test data preparation
# raw_source_test_file = r"/content/drive/My Drive/joey_experiments/data/test.ru"
# raw_target_test_file = r"/content/drive/My Drive/joey_experiments/data/test.be"

In [10]:
# source_test_file, _model = preprocess(raw_source_test_file, source_language)
# target_test_file, _model = preprocess(raw_target_test_file, target_language)

In [11]:
# ! head "$raw_source_test_file"*
# ! head "$raw_target_test_file"*

In [12]:
# import pandas as pd

# def create_df(source_f, target_f, lowercase_first_letter=True):
#    source = []
#    target = []
#    with open(source_f) as f:
#        for i, line in enumerate(f):
#          if lowercase_first_letter:
#            line = line[0].lower() + line[1:]
#          source.append(line.strip())    
   
#    with open(target_f) as f:
#        for j, line in enumerate(f):
#          if lowercase_first_letter:
#            line = line[0].lower() + line[1:]
#          target.append(line.strip())
       
#    df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

#    df_pp = df.drop_duplicates()
#    df_pp = df_pp.sample(frac=1, random_state=42).reset_index(drop=True)
#    #print(df.head(3))
#    return df_pp

In [13]:
# train_df = create_df(source_file, target_file)
# test_df = create_df(source_test_file, target_test_file)

In [14]:
# # We use 60k for dev, rest (aprx. 6k) for test
# import csv

# num_train_patterns = int(len(train_df)*0.9)

# train = train_df.head(num_train_patterns)
# dev = train_df.drop(train.index)

# def write_to_files(prefix, source):
#   with open(f"{prefix}."+source_language, "w") as src_file, open(f"{prefix}."+target_language, "w") as trg_file:
#     for index, row in source.iterrows():
#       src_file.write(row["source_sentence"]+"\n")
#       trg_file.write(row["target_sentence"]+"\n")

In [15]:
#write_to_files("train", train)
# write_to_files("dev", dev)
# write_to_files("test", test_df)

In [16]:
# ! head train.*
# ! head test.*

In [17]:
# Install JoeyNMT
%%capture
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

In [18]:
! cd joeynmt; python3 -m unittest

................................s.......................
----------------------------------------------------------------------
Ran 56 tests in 1.177s

OK (skipped=1)


In [19]:
%%capture
# ! pip install sentencepiece

In [20]:
# import sentencepiece as spm

# spm.SentencePieceTrainer.Train(f"--input=train.{source_language} --model_prefix={source_language}_sentp_{model_type} --vocab_size=16000 --character_coverage=1.0 --model_type={model_type}")
# spm.SentencePieceTrainer.Train(f"--input=train.{target_language} --model_prefix={target_language}_sentp_{model_type} --vocab_size=16000 --character_coverage=1.0 --model_type={model_type}")

# sp_source = spm.SentencePieceProcessor()
# sp_source.Load(f"{source_language}_sentp.model")

# sp_target = spm.SentencePieceProcessor()
# sp_target.Load(f"{target_language}_sentp.model")

# def encode(filename, source):
#   with open(f"{filename}.{model_type}."+source_language, "w") as src_file, open(f"{filename}.{model_type}."+target_language, "w") as trg_file:
#     for index, row in source.iterrows():
#       src_file.write("".join(sp_source.EncodeAsPieces(row["source_sentence"]))+ "\n")
#       trg_file.write("".join(sp_target.EncodeAsPieces(row["target_sentence"]))+ "\n")

# encode("train", train)
# encode("dev", dev)
# encode("test", test_df)

# from os import path

# os.environ["data_path"] = path.join("joeynmt", "data", lang_pair)

# # Create directory, move everyone we care about to the JOEY correct location
# ! mkdir -p $data_path
# ! cp train.* $data_path
# ! cp test.* $data_path
# ! cp dev.* $data_path
# ! ls $data_path

# # Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py $data_path/train.$model_type.$src $data_path/train.$model_type.$tgt --output_path $data_path/vocab.txt

In [21]:
# # Do subword NMT
# ! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 5000 -o $bpe_codes_file --write-vocabulary vocab.$src vocab.$tgt

# # Apply BPE splits to the development and test data.
# ! subword-nmt apply-bpe -c $bpe_codes_file --vocabulary vocab.$src < train.$src > train.bpe.$src
# ! subword-nmt apply-bpe -c $bpe_codes_file --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

# ! subword-nmt apply-bpe -c $bpe_codes_file --vocabulary vocab.$src < dev.$src > dev.bpe.$src
# ! subword-nmt apply-bpe -c $bpe_codes_file --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
# ! subword-nmt apply-bpe -c $bpe_codes_file --vocabulary vocab.$src < test.$src > test.bpe.$src
# ! subword-nmt apply-bpe -c $bpe_codes_file --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# # Create directory, move everyone we care about to the correct location
# ! mkdir -p $data_path
# ! cp train.* $data_path
# ! cp test.* $data_path
# ! cp dev.* $data_path
# ! cp $bpe_codes_file $data_path
# ! ls $data_path

# # Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# # Some output
# ! echo "BPE Xhosa Sentences"
# ! tail -n 5 test.bpe.$tgt
# ! echo "Combined BPE Vocab"
# ! tail -n 10 joeynmt/data/$src$tgt/vocab.txt

In [22]:
# # Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp $bpe_codes_file "$gdrive_path"
# ! ls "$gdrive_path"

In [23]:
# # This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# # (You can of course play with all the parameters if you'd like!)

# # Create the config
# config = """
# name: "{name}_transformer"

# data:
#     src: "{source_language}"
#     trg: "{target_language}"
#     train: "data/{name}/train.{model_type}"
#     dev:   "data/{name}/dev.{model_type}"
#     test:   "data/{name}/test.{model_type}"
#     level: "{model_type}"
#     lowercase: False
#     max_sent_length: 130
#     src_vocab: "data/{name}/vocab.txt"
#     trg_vocab: "data/{name}/vocab.txt"

# testing:
#     beam_size: 5
#     alpha: 1.0

# training:
#     #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
#     random_seed: 42
#     optimizer: "adam"
#     normalization: "tokens"
#     adam_betas: [0.9, 0.999] 
#     scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
#     patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
#     learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
#     learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
#     decrease_factor: 0.7
#     loss: "crossentropy"
#     learning_rate: 0.0003
#     learning_rate_min: 0.00000001
#     weight_decay: 0.0
#     label_smoothing: 0.1
#     batch_size: 4096
#     batch_type: "token"
#     eval_batch_size: 3600
#     eval_batch_type: "token"
#     batch_multiplier: 1
#     early_stopping_metric: "ppl"
#     epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
#     validation_freq: 4000          # TODO: Set to at least once per epoch.
#     logging_freq: 100
#     eval_metric: "bleu"
#     model_dir: "models/{name}_transformer_new"
#     overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
#     shuffle: True
#     use_cuda: {use_cuda}
#     max_output_length: 130
#     print_valid_sents: [0, 1, 2, 3]
#     keep_last_ckpts: 3

# model:
#     initializer: "xavier"
#     bias_initializer: "zeros"
#     init_gain: 1.0
#     embed_initializer: "xavier"
#     embed_init_gain: 1.0
#     tied_embeddings: True
#     tied_softmax: True
#     encoder:
#         type: "transformer"
#         num_layers: 6
#         num_heads: 4             # TODO: Increase to 8 for larger data.
#         embeddings:
#             embedding_dim: 256   # TODO: Increase to 512 for larger data.
#             scale: True
#             dropout: 0.2
#         # typically ff_size = 4 x hidden_size
#         hidden_size: 256         # TODO: Increase to 512 for larger data.
#         ff_size: 1024            # TODO: Increase to 2048 for larger data.
#         dropout: 0.3
#     decoder:
#         type: "transformer"
#         num_layers: 6
#         num_heads: 4              # TODO: Increase to 8 for larger data.
#         embeddings:
#             embedding_dim: 256    # TODO: Increase to 512 for larger data.
#             scale: True
#             dropout: 0.2
#         # typically ff_size = 4 x hidden_size
#         hidden_size: 256         # TODO: Increase to 512 for larger data.
#         ff_size: 1024            # TODO: Increase to 2048 for larger data.
#         dropout: 0.3
# """.format(name=lang_pair, gdrive_path=os.environ["gdrive_path"], source_language=source_language, 
#            target_language=target_language, model_type=model_type, use_cuda=use_cuda)

# with open("joeynmt/configs/transformer_{name}.yaml".format(name=lang_pair),'w') as f:
#     f.write(config)

In [24]:
# # Train the model
# # You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
# !cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [25]:
# # Copy the created models from the notebook storage to google drive for persistant storage 
# !cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer_new/"

In [26]:
# # Output our validation accuracy
# ! cat "$gdrive_path/models/${src}${tgt}_transformer_new/validations.txt"

In [27]:
!mkdir -p joeynmt/models/${src}${tgt}_transformer_new/ && cp -r "$gdrive_path/models/"* joeynmt/models/${src}${tgt}_transformer_new/
!mkdir -p joeynmt/data/${src}${tgt}/ && cp -r "$gdrive_path/data/"* joeynmt/data/${src}${tgt}/
!cp -r joeynmt/data/${src}${tgt}/{bpe.codes.5000,vocab.ru} ./

In [28]:
# # Test our model
# ! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/config.yaml" --output_path "$gdrive_path/models/predictions"

In [29]:
def interactive_translate(text):
  os.environ["config"] = "config.yaml" if use_cuda else "config_non_cuda.yaml"
  ! echo "$text" | sacremoses tokenize | sacremoses truecase -m "$source_tc_model" | subword-nmt apply-bpe -c "$bpe_codes_file" --vocabulary vocab.$source_language > "joeynmt/in.txt"
  ! cd joeynmt; python3 -m joeynmt translate "$gdrive_path/models/$config" < in.txt 2> /dev/null | sacremoses detruecase 2> /dev/null | sacremoses detokenize 2> /dev/null | sed "s/ '/'/"

In [30]:
interactive_translate("Пусть мой комментарий затеряется, пусть утонет среди других.")

Няхай мой каментарый загубляецца, няхай патанне сярод іншых.


In [31]:
#interactive_translate("В Витебске кратковременный снег, мокрый снег.")
interactive_translate("В дневные часы 15 марта кое-где отмечались дожди.")

У дзённыя гадзіны 15 сакавіка я-дзе адзначаліся дажджы.


In [32]:
interactive_translate("Как объяснили медработники, сделано для того, чтобы защитить стены от возможных ударов каталок.")
interactive_translate("Также на первом этаже расположены различные диагностические службы.")
interactive_translate("Например кабинет компьютерной томографии.")
interactive_translate("Аппарат, с помощью которого проводят это информативное высокотехнологичное исследование, самый современный.")
interactive_translate("Рядом - изотопная лаборатория, где выполняют радионуклидную диагностику.")

Як растлумачылі медработнікі, зроблена для таго, каб абараніць сцены ад магчымых удараў каталок.
Таксама на першым паверсе размешчаны розныя дыягнастычныя службы.
Напрыклад кабінет камп'ютарнай тамаграфіі.
Апарат, з дапамогай якога праводзяць гэта інфарматыўнае высокатэхналагічнае даследаванне, самы сучасны.
Побач — ізатопная лабараторыя, дзе выконваюць радыёнуклідную дыягнастыку.


In [33]:
# def file_translate(file):
#   os.environ["config"] = "config.yaml" if use_cuda else "config_non_cuda.yaml"
#   ! sacremoses tokenize < "$file" | sacremoses truecase -m "$source_tc_model" | subword-nmt apply-bpe -c "$bpe_codes_file" --vocabulary vocab.$source_language > "joeynmt/in.txt"
#   ! cd joeynmt; python3 -m joeynmt translate "$gdrive_path/models/$config" < in.txt 2> /dev/null | sacremoses detruecase 2> /dev/null | sacremoses detokenize 2> /dev/null | sed "s/ '/'/" > joey_pred.txt

In [34]:
# folder = f'{gdrive_path}/data/data_test/TestDatasets/AntonTestData'
# in_file_name = "2016-03_ru.txt"
# in_filepath = f'{folder}/{in_file_name}'
# file_translate(in_filepath)

In [35]:
# !cp -r joeynmt/joey_pred.txt "$folder"
# !cp -r joeynmt/in.txt "$folder"

In [36]:
# folder = f'{gdrive_path}/data/data_test/TestDatasets/TsimafeiMiddleCorpusTest'
# in_file_name = "MiddleCorpusTestPart_Rus.txt"
# in_filepath = f'{folder}/{in_file_name}'
# file_translate(in_filepath)

In [37]:
# !cp -r joeynmt/joey_pred.txt "$folder"
# !cp -r joeynmt/in.txt "$folder"